In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np
#pandas- librărie pentru lucrul cu fișierele
import pandas as pd
import torch
import torch.nn as nn

In [248]:
from google.colab import files
uploaded = files.upload()

Saving pima-indians-diabetes.csv to pima-indians-diabetes (5).csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['pima-indians-diabetes.csv']))

In [250]:
df.values

array([[1.00e+00, 8.50e+01, 6.60e+01, ..., 3.51e-01, 3.10e+01, 0.00e+00],
       [8.00e+00, 1.83e+02, 6.40e+01, ..., 6.72e-01, 3.20e+01, 1.00e+00],
       [1.00e+00, 8.90e+01, 6.60e+01, ..., 1.67e-01, 2.10e+01, 0.00e+00],
       ...,
       [5.00e+00, 1.21e+02, 7.20e+01, ..., 2.45e-01, 3.00e+01, 0.00e+00],
       [1.00e+00, 1.26e+02, 6.00e+01, ..., 3.49e-01, 4.70e+01, 1.00e+00],
       [1.00e+00, 9.30e+01, 7.00e+01, ..., 3.15e-01, 2.30e+01, 0.00e+00]])

In [0]:
#Dataset - o clasă din PyTorch foarte utilă gestionării seturilor de date
class PimaDiabetesDataset(Dataset):
  # Initialize your data, download, etc.
  def __init__(self):
    #Citim setul de date
    df = pd.read_csv(io.BytesIO(uploaded['pima-indians-diabetes.csv']), header=None, dtype=np.float32)
    xy = torch.from_numpy(df.values)
    self.len = xy.shape[0]
    #Vom folosi ca input toate valorile mai puțin ultima coloană
    self.x_data = xy[:, 0:-1]
    #Vom folosi ca output ultima coloană
    self.y_data = xy[:, [-1]]
  
  def __getitem__(self, index):
     return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

In [0]:
dataset = PimaDiabetesDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=1)

In [253]:
dataset[0]

(tensor([  6.0000, 148.0000,  72.0000,  35.0000,   0.0000,  33.6000,   0.6270,
          50.0000]), tensor([1.]))

In [0]:
class Model(nn.Module):

  def __init__(self):
    #In the constructor we instantiate two nn.Linear module
    super(Model, self).__init__()
    self.l1 = nn.Linear(8, 6)
    self.l2 = nn.Linear(6, 4)
    self.l3 = nn.Linear(4, 1)

    self.sigmoid = nn.Sigmoid()
    self.ReLU = nn.ReLU()

  def forward(self, x):
    # In the forward function we accept a Variable of input data and we must return
    # a Variable of output data. We can use Modules defined in the constructor as
    # well as arbitrary operators on Variables.
    out1 = self.ReLU(self.l1(x))
    out2 = self.ReLU(self.l2(out1))
    y_pred = self.sigmoid(self.l3(out2))
    return y_pred

In [0]:
model = Model()

In [0]:
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

In [257]:
# Training loop
for epoch in range(200):
  # Aplicăm modificarea de learning rate per epocă, deci va trebui să avem un istoric al loss-ului
  losses=[]

  for i, data in enumerate(train_loader, 0):
    # get the inputs
    inputs, labels = data

    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(inputs)

    # Compute and print loss
    loss = criterion(y_pred, labels)
    print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')

    losses.append(loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    mean_loss=sum(losses)/len(losses)
    scheduler.step(mean_loss)

Epoch 16 | Batch: 310 | Loss: 0.0000
Epoch 16 | Batch: 311 | Loss: 0.0000
Epoch 16 | Batch: 312 | Loss: 0.0000
Epoch 16 | Batch: 313 | Loss: 100.0000
Epoch 16 | Batch: 314 | Loss: 77.6197
Epoch 16 | Batch: 315 | Loss: 0.0000
Epoch 16 | Batch: 316 | Loss: 100.0000
Epoch 16 | Batch: 317 | Loss: 100.0000
Epoch 16 | Batch: 318 | Loss: 0.0000
Epoch 16 | Batch: 319 | Loss: 0.0000
Epoch 16 | Batch: 320 | Loss: 0.0000
Epoch 16 | Batch: 321 | Loss: 0.0000
Epoch 16 | Batch: 322 | Loss: 0.0000
Epoch 16 | Batch: 323 | Loss: 0.0000
Epoch 16 | Batch: 324 | Loss: 72.8255
Epoch 16 | Batch: 325 | Loss: 0.0000
Epoch 16 | Batch: 326 | Loss: 0.0000
Epoch 16 | Batch: 327 | Loss: 100.0000
Epoch 16 | Batch: 328 | Loss: 0.0000
Epoch 16 | Batch: 329 | Loss: 0.0000
Epoch 16 | Batch: 330 | Loss: 0.0000
Epoch 16 | Batch: 331 | Loss: 0.0000
Epoch 16 | Batch: 332 | Loss: 0.0000
Epoch 16 | Batch: 333 | Loss: 0.0000
Epoch 16 | Batch: 334 | Loss: 0.0000
Epoch 16 | Batch: 335 | Loss: 100.0000
Epoch 16 | Batch: 336 | Lo

KeyboardInterrupt: ignored